In [6]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.fits as pf
import sys, glob

#thedir='/project/bootcamp/cslage/ats_data/20190416/ptc_kirk_config/' # Current voltages
thedir='/project/bootcamp/cslage/ats_data/20190416/ptc_craig_config_lights_off/' # BNL/ITL voltages
%cd $thedir


/project/bootcamp/cslage/ats_data/20190416/ptc_craig_config_lights_off


In [7]:
files = glob.glob('AT_C_20190416_000???_exp_*0.fits')
name = files[12][25:27]
print(files)
print(name)


['AT_C_20190416_000047_exp_10.fits', 'AT_C_20190416_000056_exp_50.fits', 'AT_C_20190416_000059_exp_70.fits', 'AT_C_20190416_000058_exp_60.fits', 'AT_C_20190416_000052_exp_30.fits', 'AT_C_20190416_000054_exp_40.fits', 'AT_C_20190416_000049_exp_20.fits', 'AT_C_20190416_000045_exp_0.fits', 'AT_C_20190416_000061_exp_80.fits', 'AT_C_20190416_000062_exp_80.fits', 'AT_C_20190416_000046_exp_0.fits', 'AT_C_20190416_000063_exp_90.fits', 'AT_C_20190416_000053_exp_40.fits', 'AT_C_20190416_000048_exp_10.fits', 'AT_C_20190416_000060_exp_70.fits', 'AT_C_20190416_000066_exp_100.fits', 'AT_C_20190416_000057_exp_60.fits', 'AT_C_20190416_000064_exp_90.fits', 'AT_C_20190416_000050_exp_20.fits', 'AT_C_20190416_000051_exp_30.fits', 'AT_C_20190416_000055_exp_50.fits', 'AT_C_20190416_000065_exp_100.fits']
40


In [8]:
xstart = 505
xstop = 542
ov_start = 512
ystart = 200
ystop = 1800
xaxis = np.linspace(xstart,xstop-1,xstop-xstart)

hdulist = pf.open(files[0], mode='readonly', do_not_scale_image_data=True)
hdr18=hdulist[18].header
Vph = hdr18['PCK_HIP']
Vpl = hdr18['PCK_LOWP']
Vsh = hdr18['SCK_HIP']
Vsl = hdr18['SCK_LOWP']
Vog = hdr18['OGOV']
Vbb = hdr18['HVBIAS_V']

for file in [files[12]]:
    print(file)
    name = file[25:27]
    hdulist = pf.open(file, mode='readonly', do_not_scale_image_data=True)
    plt.figure(figsize=(15,15))
    plt.subplots_adjust(hspace = 0.5, wspace = 0.5)
    plt.suptitle("Serial CTE by segment - ITL AuxTel CCD 098\n Vph=%.2f; Vpl=%.2f; Vsh=%.2f; Vsl=%.2f; Vog=%.2f; Vbb=%.2f"%(Vph,Vpl,Vsh,Vsl,Vog,Vbb), fontsize = 16)
    for segment in range(16):
            #try:
            plt.subplot(4,4,segment+1)
            imhdr=hdulist[segment+1].header
            extname = imhdr['EXTNAME']
            plt.title("CTE - %s"%extname)
            data = np.array(hdulist[segment+1].data + 32768, dtype = np.int32)
            flat_overscan = np.mean(np.array(data[:,xstop-8:xstop]),axis = 1)
            cte_data = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].mean(axis=0))[xstart:xstop]
            cte_std = ((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,:].std(axis=0) / np.sqrt(float(ystop-ystart)))[xstart:xstop]
            cti = np.median((np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start]\
            / (np.transpose(np.transpose(data) - flat_overscan))[ystart:ystop,ov_start-1]) / ov_start

            plt.scatter(xaxis, cte_data, color = 'blue')
            plt.errorbar(xaxis, cte_data, yerr=cte_std)
            plt.text(xstart + 8, 20000, "CTI = %.2E"%cti, fontsize = 12)
            plt.yscale('log',nonposy='clip')
            plt.ylim(.01, 200000)
            plt.xticks([510,520,530,540])
            plt.xlabel("X (First overscan = 512)")
            plt.ylabel("Signal(ADU)")
            print("Segment %s done"%extname)
            #except:
            #continue
    plt.savefig(thedir+"Serial_CTE_%s_16Apr19.pdf"%name)
    plt.clf()


AT_C_20190416_000053_exp_40.fits
Segment Segment10 done
Segment Segment11 done
Segment Segment12 done
Segment Segment13 done
Segment Segment14 done
Segment Segment15 done
Segment Segment16 done
Segment Segment17 done
Segment Segment07 done
Segment Segment06 done
Segment Segment05 done
Segment Segment04 done
Segment Segment03 done
Segment Segment02 done
Segment Segment01 done
Segment Segment00 done
